In [1]:
# Kunci jawaban yang benar untuk nomor 1 sampai 20 diambil dari input pengguna
correct_answers = {}
for question_number in range(1, 21):
    while True:
        correct_answer = input(f"Please enter the correct answer for question number {question_number}: ").strip()
        if correct_answer.upper() in ['A', 'B', 'C', 'D', 'E']:
            break
        else:
            print("Invalid input. Please enter 'A', 'B', 'C', 'D', or 'E'.")
    correct_answers[question_number] = correct_answer.upper()

#print semua inputan
print(correct_answers)


{1: 'A', 2: 'A', 3: 'C', 4: 'B', 5: 'A', 6: 'C', 7: 'B', 8: 'A', 9: 'D', 10: 'B', 11: 'B', 12: 'C', 13: 'B', 14: 'A', 15: 'B', 16: 'D', 17: 'B', 18: 'A', 19: 'B', 20: 'A'}


In [2]:
import cv2
import imutils
from skimage.filters import threshold_local
from transform import perspective_transform

original_img = cv2.imread('Normal1.jpg')
copy = original_img.copy()



# The resized height in hundreds
ratio = original_img.shape[0] / 500.0
img_resize = imutils.resize(original_img, height=500)

gray_image = cv2.cvtColor(img_resize, cv2.COLOR_BGR2GRAY)
blurred_image = cv2.GaussianBlur(gray_image, (3, 3), 0)
edged_img = cv2.Canny(gray_image, 75, 200)

cnts, _ = cv2.findContours(edged_img, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
cnts = sorted(cnts, key=cv2.contourArea, reverse=True)[:5]

# Make sure 'doc' is defined before using it
for c in cnts:
    peri = cv2.arcLength(c, True)
    approx = cv2.approxPolyDP(c, 0.02 * peri, True)

    if len(approx) == 4:
        doc = approx
        break

warped_image = perspective_transform(copy, doc.reshape(4, 2) * ratio)
warped_image = cv2.cvtColor(warped_image, cv2.COLOR_BGR2GRAY)

T = threshold_local(warped_image, 11, offset=10, method="gaussian")
warped = (warped_image > T).astype("uint8") * 255

# Resize the processed image to a fixed size of 500x500 pixels
warped_resized_fixed = cv2.resize(warped, (898, 1223))
cv2.imwrite('./'+'scan'+'.jpg',warped_resized_fixed)

# Display the final scanned image
cv2.imshow("Final Scanned image", warped_resized_fixed)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [3]:
import cv2
import numpy as np

# Baca citra
image = cv2.imread('scan.jpg')
# Koordinat ROI: (x_start, y_start, width, height)
roi_coords = (30, 224, 837, 218)
# Tentukan ROI menggunakan koordinat yang sudah diberikan
x_start, y_start, width, height = roi_coords
roi = image[y_start:y_start+height, x_start:x_start+width]

# Konversi ROI ke grayscale
roi_gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)

# Terapkan teknik binarisasi menggunakan metode Otsu
_, binary_image = cv2.threshold(roi_gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

# Koordinat awal dan ukuran untuk nomor 1 sampai 10
x_start_A_1_10, y_start_A_1_10 = 40, 2
width_1_10, height_1_10 = 72, 21
distance_between_choices_1_10 = 72
distance_between_rows_1_10 = 21

# Koordinat awal dan ukuran untuk nomor 11 sampai 20
x_start_A_11_20, y_start_A_11_20 = 476, 1
width_11_20, height_11_20 = 71, 21
distance_between_choices_11_20 = 71
distance_between_rows_11_20 = 21

# Fungsi untuk menganalisis jawaban yang dipilih
def analyze_choices(image, x_start, y, width, height, distance_between_choices):
    choices = ['A', 'B', 'C', 'D', 'E']
    black_pixels_count = []
    for choice in choices:
        x = x_start + (choices.index(choice) * distance_between_choices)
        cropped_image = image[y:y+height, x:x+width]
        # Hitung jumlah piksel hitam (piksel dengan nilai 0)
        black_pixels = np.sum(cropped_image == 0)
        black_pixels_count.append(black_pixels)
    # Tentukan pilihan dengan jumlah piksel hitam terbanyak
    max_black_pixels = max(black_pixels_count)
    selected_choice = choices[black_pixels_count.index(max_black_pixels)]
    return selected_choice

# Analisis jawaban peserta untuk nomor pertanyaan 1 sampai 20
peserta_answers = {}
for question_number in range(1, 21):
    if question_number <= 10:
        x_start = x_start_A_1_10
        y_start = y_start_A_1_10 + (question_number - 1) * distance_between_rows_1_10
        width = width_1_10
        height = height_1_10
        distance_between_choices = distance_between_choices_1_10
    else:
        x_start = x_start_A_11_20
        y_start = y_start_A_11_20 + (question_number - 11) * distance_between_rows_11_20
        width = width_11_20
        height = height_11_20
        distance_between_choices = distance_between_choices_11_20
    selected_choice = analyze_choices(binary_image, x_start, y_start, width, height, distance_between_choices)
    peserta_answers[question_number] = selected_choice
    correct_answer = correct_answers[question_number]
    if selected_choice:
        print(f"Peserta memilih jawaban: {selected_choice} pada nomor {question_number}")
        if selected_choice == correct_answer:
            print("Jawaban peserta sudah benar.")
        else:
            print(f"Jawaban peserta salah. Jawaban yang benar adalah {correct_answer}.")
    else:
        print(f"Tidak ada jawaban yang dipilih pada nomor {question_number}")

# Fungsi untuk menghitung skor peserta
def calculate_score(peserta_answers, correct_answers):
    score = 0
    for question_number, peserta_answer in peserta_answers.items():
        if peserta_answer == correct_answers[question_number]:
            score += 1
    return score

# Fungsi untuk menghitung skor peserta
def calculate_score(peserta_answers, correct_answers):
    score = 0
    for question_number, peserta_answer in peserta_answers.items():
        if peserta_answer == correct_answers[question_number]:
            score += 1
    return score

# Hitung dan tampilkan skor peserta untuk nomor 1 
peserta_score = calculate_score(peserta_answers, correct_answers)
print(f"Skor peserta untuk nomor 1 sampai 20 adalah: {peserta_score} dari total {len(correct_answers)} pertanyaan.") 


Peserta memilih jawaban: A pada nomor 1
Jawaban peserta sudah benar.
Peserta memilih jawaban: A pada nomor 2
Jawaban peserta sudah benar.
Peserta memilih jawaban: C pada nomor 3
Jawaban peserta sudah benar.
Peserta memilih jawaban: B pada nomor 4
Jawaban peserta sudah benar.
Peserta memilih jawaban: A pada nomor 5
Jawaban peserta sudah benar.
Peserta memilih jawaban: C pada nomor 6
Jawaban peserta sudah benar.
Peserta memilih jawaban: B pada nomor 7
Jawaban peserta sudah benar.
Peserta memilih jawaban: A pada nomor 8
Jawaban peserta sudah benar.
Peserta memilih jawaban: D pada nomor 9
Jawaban peserta sudah benar.
Peserta memilih jawaban: B pada nomor 10
Jawaban peserta sudah benar.
Peserta memilih jawaban: B pada nomor 11
Jawaban peserta sudah benar.
Peserta memilih jawaban: C pada nomor 12
Jawaban peserta sudah benar.
Peserta memilih jawaban: B pada nomor 13
Jawaban peserta sudah benar.
Peserta memilih jawaban: A pada nomor 14
Jawaban peserta sudah benar.
Peserta memilih jawaban: B pa

In [4]:
import cv2
import pytesseract

# Set the path to the Tesseract executable
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

# Koordinat awal dan ukuran untuk nomor 1 sampai 10
x_start_A_1_10, y_start_A_1_10 = 40, 2
width_1_10, height_1_10 = 72, 21
distance_between_choices_1_10 = 73  # Jarak antara pilihan A ke B, B ke C, dan seterusnya
distance_between_rows_1_10 = 21  # Jarak antar baris jawaban

# Koordinat awal dan ukuran untuk nomor 11 sampai 20
x_start_A_11_20, y_start_A_11_20 = 476, 1
width_11_20, height_11_20 = 71, 21
distance_between_choices_11_20 = 72  # Jarak antara pilihan A ke B, B ke C, dan seterusnya
distance_between_rows_11_20 = 21  # Jarak antar baris jawaban


# Fungsi untuk menganalisis jawaban yang dipilih berdasarkan kemiripan dengan alfabet
def analyze_choices_by_similarity(image, x_start, y, width, height, distance_between_choices):
    choices = ['A', 'B', 'C', 'D', 'E']
    similarity_scores = []

    for choice in choices:
        x = x_start + (choices.index(choice) * distance_between_choices)

        # Ensure the cropped region does not go beyond image boundaries
        if x < 0 or x + width > image.shape[1]:
            continue

        cropped_image = image[y:y+height, x:x+width]
        # Gunakan OCR untuk mengenali karakter pada area yang di-crop
        recognized_text = pytesseract.image_to_string(cropped_image, config='--psm 10')
        # Hitung skor kemiripan antara teks yang dikenali dengan alfabet yang seharusnya
        similarity_score = 1 - (levenshtein_distance(recognized_text.strip(), choice) / len(choice))
        similarity_scores.append(similarity_score)

    # Jika tidak ada pilihan yang valid, kembalikan None
    if not similarity_scores:
        return None

    # Tentukan pilihan dengan skor kemiripan terendah (paling tidak mirip)
    min_similarity_score = min(similarity_scores)
    selected_choice = choices[similarity_scores.index(min_similarity_score)]
    return selected_choice

# Fungsi untuk menghitung jarak Levenshtein (jarak edit)
def levenshtein_distance(s1, s2):
    if len(s1) < len(s2):
        return levenshtein_distance(s2, s1)
    if len(s2) == 0:
        return len(s1)
    previous_row = range(len(s2) + 1)
    for i, c1 in enumerate(s1):
        current_row = [i + 1]
        for j, c2 in enumerate(s2):
            insertions = previous_row[j + 1] + 1
            deletions = current_row[j] + 1
            substitutions = previous_row[j] + (c1 != c2)
            current_row.append(min(insertions, deletions, substitutions))
        previous_row = current_row
    return previous_row[-1]

# Baca gambar lembar jawaban
image = cv2.imread("hasil_binarisasi.jpg", cv2.IMREAD_GRAYSCALE)

# Analisis jawaban peserta untuk nomor pertanyaan 1 sampai 20
peserta_answers = {}
for question_number in range(1, 21):
    if question_number <= 10:
        x_start = x_start_A_1_10
        y_start = y_start_A_1_10 + (question_number - 1) * distance_between_rows_1_10
        width = width_1_10
        height = height_1_10
        distance_between_choices = distance_between_choices_1_10
    else:
        x_start = x_start_A_11_20
        y_start = y_start_A_11_20 + (question_number - 11) * distance_between_rows_11_20
        width = width_11_20
        height = height_11_20
        distance_between_choices = distance_between_choices_11_20
    
    selected_choice = analyze_choices_by_similarity(image, x_start, y_start, width, height, distance_between_choices)
    peserta_answers[question_number] = selected_choice
    correct_answer = correct_answers[question_number]
    if selected_choice:
        print(f"Peserta memilih jawaban: {selected_choice} pada nomor {question_number}")
        if selected_choice == correct_answer:
            print("Jawaban peserta sudah benar.")
        else:
            print(f"Jawaban peserta salah. Jawaban yang benar adalah {correct_answer}.")
    else:
        print(f"Tidak ada jawaban yang dipilih pada nomor {question_number}")

# Fungsi untuk menghitung skor peserta
def calculate_score(peserta_answers, correct_answers):
    score = 0
    for question_number, peserta_answer in peserta_answers.items():
        if peserta_answer == correct_answers[question_number]:
            score += 1
    return score

# Hitung dan tampilkan skor peserta untuk nomor 1 sampai 20
peserta_score = calculate_score(peserta_answers, correct_answers)
print(f"Skor peserta untuk nomor 1 sampai 20 adalah: {peserta_score} dari total {len(correct_answers)} pertanyaan.")


Peserta memilih jawaban: B pada nomor 1
Jawaban peserta salah. Jawaban yang benar adalah A.
Peserta memilih jawaban: B pada nomor 2
Jawaban peserta salah. Jawaban yang benar adalah A.
Peserta memilih jawaban: E pada nomor 3
Jawaban peserta salah. Jawaban yang benar adalah C.
Peserta memilih jawaban: B pada nomor 4
Jawaban peserta sudah benar.
Peserta memilih jawaban: C pada nomor 5
Jawaban peserta salah. Jawaban yang benar adalah A.
Peserta memilih jawaban: C pada nomor 6
Jawaban peserta sudah benar.
Peserta memilih jawaban: D pada nomor 7
Jawaban peserta salah. Jawaban yang benar adalah B.
Peserta memilih jawaban: B pada nomor 8
Jawaban peserta salah. Jawaban yang benar adalah A.
Peserta memilih jawaban: B pada nomor 9
Jawaban peserta salah. Jawaban yang benar adalah D.
Peserta memilih jawaban: E pada nomor 10
Jawaban peserta salah. Jawaban yang benar adalah B.
Peserta memilih jawaban: B pada nomor 11
Jawaban peserta sudah benar.
Peserta memilih jawaban: B pada nomor 12
Jawaban pesert

# haha

In [37]:
# import cv2

# # Inisialisasi koordinat global untuk titik awal dan akhir
# x_start, y_start, x_end, y_end = 0, 0, 0, 0
# cropping = False

# # Fungsi callback untuk event mouse
# def mouse_crop(event, x, y, flags, param):
#     global x_start, y_start, x_end, y_end, cropping

#     # Jika mouse ditekan, ambil titik awal dan mulai cropping
#     if event == cv2.EVENT_LBUTTONDOWN:
#         x_start, y_start, x, y = x, y, x, y
#         cropping = True

#     # Jika mouse dilepas, ambil titik akhir dan selesaikan cropping
#     elif event == cv2.EVENT_LBUTTONUP:
#         x_end, y_end = x, y
#         cropping = False # Cropping selesai

#         # Tampilkan koordinat ROI dan gambar yang di-crop
#         roi = (x_start, y_start, x_end-x_start, y_end-y_start)
#         print(f"Koordinat ROI: {roi}")
#         cv2.rectangle(image, (x_start, y_start), (x_end, y_end), (255, 0, 0), 2)
#         cv2.imshow("Image", image)

# # Baca gambar
# image = cv2.imread('scan.png')
# if image is None:
#     print("Error: Tidak dapat membaca gambar. Periksa path file gambar.")
# else:
#     # Buat jendela dan tetapkan fungsi callback untuk event mouse
#     cv2.namedWindow("Image")
#     cv2.setMouseCallback("Image", mouse_crop)

#     # Loop utama untuk menampilkan gambar dan menunggu input mouse
#     while True:
#         i = image.copy()
#         if not cropping:
#             cv2.imshow("Image", image)
#         elif cropping:
#             cv2.rectangle(i, (x_start, y_start), (x_end, y_end), (255, 0, 0), 2)
#             cv2.imshow("Image", i)
#         cv2.waitKey(1)

#         # Keluar dari loop jika ditekan tombol 'ESC'
#         if cv2.waitKey(20) == 27:
#             break

#     # Tutup semua jendela yang terbuka
#     cv2.destroyAllWindows()


# cp

In [38]:
# correct_answers = {
#     1: 'B', 2: 'C', 3: 'D', 4: 'E', 5: 'A',
#     6: 'C', 7: 'D', 8: 'E', 9: 'D', 10: 'A',
#     11: 'A', 12: 'B', 13: 'C', 14: 'D', 15: 'E',
#     16: 'A', 17: 'B', 18: 'C', 19: 'D', 20: 'E'
#     1: 'A', 2: 'B', 3: 'A', 4: 'B', 5: 'A',
#     6: 'B', 7: 'A', 8: 'B', 9: 'A', 10: 'B',
#     11: 'C', 12: 'D', 13: 'C', 14: 'D', 15: 'C',
#     16: 'D', 17: 'C', 18: 'D', 19: 'C', 20: 'D'
#     1: 'A', 2: 'B', 3: 'C', 4: 'D', 5: 'E',
#     6: 'D', 7: 'C', 8: 'B', 9: 'A', 10: 'B',
#     11: 'A', 12: 'B', 13: 'C', 14: 'D', 15: 'E',
#     16: 'D', 17: 'C', 18: 'B', 19: 'A', 20: 'B'
#     1: 'B', 2: 'D', 3: 'C', 4: 'B', 5: 'C',
#     6: 'D', 7: 'A', 8: 'B', 9: 'C', 10: 'D',
#     11: 'A', 12: 'B', 13: 'C', 14: 'D', 15: 'E',
#     16: 'D', 17: 'C', 18: 'B', 19: 'A', 20: 'B'
# }


In [39]:
# import cv2
# import numpy as np

# # Baca citra
# image = cv2.imread('scan.png')

# # Koordinat ROI: (x_start, y_start, width, height)
# roi_coords = (30, 224, 837, 218)
# # Tentukan ROI menggunakan koordinat yang sudah diberikan
# x_start, y_start, width, height = roi_coords
# roi = image[y_start:y_start+height, x_start:x_start+width]

# # Simpan hasil ROI ke dalam file gambar
# cv2.imwrite("hasil_roi.jpg", roi)

# # Baca hasil ROI dari file gambar
# roi = cv2.imread("hasil_roi.jpg", cv2.IMREAD_GRAYSCALE)

# # Terapkan teknik binarisasi menggunakan metode Otsu
# _, binary_image = cv2.threshold(roi, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

# # Simpan hasil binarisasi ke dalam file gambar
# cv2.imwrite("hasil_binarisasi.jpg", binary_image)

# # Baca gambar hasil binarisasi untuk dianalisis
# image_to_analyze = cv2.imread("hasil_binarisasi.jpg", cv2.IMREAD_GRAYSCALE)

# # Koordinat awal dan ukuran untuk nomor 1 sampai 10
# x_start_A_1_10, y_start_A_1_10 = 40, 2
# width_1_10, height_1_10 = 72, 21
# distance_between_choices_1_10 = 72
# distance_between_rows_1_10 = 21

# # Koordinat awal dan ukuran untuk nomor 11 sampai 20
# x_start_A_11_20, y_start_A_11_20 = 476, 1
# width_11_20, height_11_20 = 71, 21
# distance_between_choices_11_20 = 71
# distance_between_rows_11_20 = 21

# # Fungsi untuk menganalisis jawaban yang dipilih
# def analyze_choices(image, x_start, y, width, height, distance_between_choices):
#     choices = ['A', 'B', 'C', 'D', 'E']
#     black_pixels_count = []
#     for choice in choices:
#         x = x_start + (choices.index(choice) * distance_between_choices)
#         cropped_image = image[y:y+height, x:x+width]
#         # Hitung jumlah piksel hitam (piksel dengan nilai 0)
#         black_pixels = np.sum(cropped_image == 0)
#         black_pixels_count.append(black_pixels)
#     # Tentukan pilihan dengan jumlah piksel hitam terbanyak
#     max_black_pixels = max(black_pixels_count)
#     selected_choice = choices[black_pixels_count.index(max_black_pixels)]
#     return selected_choice

# # Analisis jawaban peserta untuk nomor pertanyaan 1 sampai 20
# peserta_answers = {}
# for question_number in range(1, 21):
#     if question_number <= 10:
#         x_start = x_start_A_1_10
#         y_start = y_start_A_1_10 + (question_number - 1) * distance_between_rows_1_10
#         width = width_1_10
#         height = height_1_10
#         distance_between_choices = distance_between_choices_1_10
#     else:
#         x_start = x_start_A_11_20
#         y_start = y_start_A_11_20 + (question_number - 11) * distance_between_rows_11_20
#         width = width_11_20
#         height = height_11_20
#         distance_between_choices = distance_between_choices_11_20
#     selected_choice = analyze_choices(image_to_analyze, x_start, y_start, width, height, distance_between_choices)
#     peserta_answers[question_number] = selected_choice

# # Fungsi untuk menghitung skor peserta
# def calculate_score(peserta_answers, correct_answers):
#     score = 0
#     for question_number, peserta_answer in peserta_answers.items():
#         if peserta_answer == correct_answers[question_number]:
#             score += 1
#     return score

# # Hitung dan tampilkan skor peserta untuk nomor 1 
# peserta_score = calculate_score(peserta_answers, correct_answers)
# print(f"Skor peserta untuk nomor 1 sampai 20 adalah: {peserta_score} dari total {len(correct_answers)} pertanyaan.") 
